In [1]:
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('data/cleaned/ga_hta_socioeconomic_2024_clean.csv')
df['language'] = LabelEncoder().fit_transform(df['language'])
df = df.dropna().set_index('postcode')

# Split by canton (80% cantons to train, rest test)
canton_list = df['canton'].unique()
train_cantons = np.random.choice(canton_list, int(0.8 * len(canton_list)), replace=False)
test_cantons = [c for c in canton_list if c not in train_cantons]

# to ensure consistency in production
train_cantons = ['SG', 'BL', 'VS', 'OW', 'NE', 'GE', 'TG', 'BE', 'SH', 'LU', 'AG',
       'UR', 'GL', 'ZG', 'SO', 'AI', 'GR', 'TI', 'BS', 'SZ']
test_cantons = ['VD', 'FR', 'JU', 'NW', 'ZH', 'AR']

train_df = df[df['canton'].isin(train_cantons)]
test_df = df[df['canton'].isin(test_cantons)]


In [2]:
# option 1 - add location effect
X_cols_loc = [
    'num_stops', 'stop_density', 'avg_daily_frequency', 'max_stop_frequency',
    'population', 'male_population', 'swiss_citizen', 'married', 'age_20_64',
    'gdp_per_capita', 'eco_activity_rate', 'unemploy_rate',
    'total_residential_housing', 'pure_residential_single', 'pure_residential_multiple',
    'language',
    'lat', 'lon'
]

X_train, y_train = train_df[X_cols_loc], train_df[['GA_pct', 'HTA_pct']]
X_test, y_test = test_df[X_cols_loc], test_df[['GA_pct', 'HTA_pct']]

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = y_train
y_test_scaled = y_test

In [3]:
def evaluate(model, X_test, y_test, name):
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred, multioutput='uniform_average')
    mae = mean_absolute_error(y_test, y_pred)
    rmse = math.sqrt(mean_squared_error(y_test, y_pred))
    print(f"{name} R²: {r2:.4f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}")

In [4]:
# linear regression
lrg = LinearRegression()
lrg.fit(X_train_scaled, y_train_scaled)
evaluate(lrg, X_test_scaled, y_test_scaled, "LinearRegressor")

LinearRegressor R²: 0.2287, MAE: 4.05, RMSE: 6.05


In [5]:
# catboost
cat_model = MultiOutputRegressor(CatBoostRegressor(verbose=0, random_state=88))
cat_model.fit(X_train_scaled, y_train_scaled)
evaluate(cat_model, X_test_scaled, y_test_scaled, "CatBoost")

CatBoost R²: 0.2880, MAE: 4.03, RMSE: 6.02


In [6]:
# option 2 - add neighboring features

In [7]:

exclude_cols = ['GA_pct', 'HTA_pct', 'GA_log', 'HTA_log', 'language', 'canton', 'neighbors']

columns_to_process = [col for col in train_df.columns
                     if col not in exclude_cols and np.issubdtype(train_df[col].dtype, np.number)]

for idx, row in train_df.iterrows():
    neighbors = eval(row['neighbors'])
    if isinstance(neighbors, list):  # Check if neighbors is a list
        neighbor_values = {col: [] for col in columns_to_process}
        # Collect values from valid neighbors
        for neighbor in neighbors:
            if int(neighbor) in train_df.index:
                neighbor_row = train_df.loc[int(neighbor)]
                for col in columns_to_process:
                    value = neighbor_row[col]
                    if not np.isnan(value):  # Skip NaN values
                        neighbor_values[col].append(value)

        # Calculate means and create new columns
        for col in columns_to_process:
            if neighbor_values[col]:  # If we have at least one valid neighbor value
                new_col_name = f"{col}_neighbor"
                train_df.at[idx, new_col_name] = np.mean(neighbor_values[col])
            else:
                new_col_name = f"{col}_neighbor"
                train_df.at[idx, new_col_name] = np.nan

In [8]:
exclude_cols = ['GA_pct', 'HTA_pct', 'GA_log', 'HTA_log', 'language', 'canton', 'neighbors']

columns_to_process = [col for col in test_df.columns
                     if col not in exclude_cols and np.issubdtype(test_df[col].dtype, np.number)]

for idx, row in test_df.iterrows():
    neighbors = eval(row['neighbors'])
    if isinstance(neighbors, list):  # Check if neighbors is a list
        neighbor_values = {col: [] for col in columns_to_process}
        # Collect values from valid neighbors
        for neighbor in neighbors:
            if int(neighbor) in test_df.index:
                neighbor_row = test_df.loc[int(neighbor)]
                for col in columns_to_process:
                    value = neighbor_row[col]
                    if not np.isnan(value):  # Skip NaN values
                        neighbor_values[col].append(value)

        # Calculate means and create new columns
        for col in columns_to_process:
            if neighbor_values[col]:  # If we have at least one valid neighbor value
                new_col_name = f"{col}_neighbor"
                test_df.at[idx, new_col_name] = np.mean(neighbor_values[col])
            else:
                new_col_name = f"{col}_neighbor"
                test_df.at[idx, new_col_name] = np.nan

In [9]:
test_df

,GA,HTA,num_stops,area_km2,avg_daily_frequency,total_daily_frequency,max_stop_frequency,neighbors,region,stop_density,...,male_population_neighbor,swiss_citizen_neighbor,married_neighbor,age_20_64_neighbor,total_residential_housing_neighbor,pure_residential_single_neighbor,pure_residential_multiple_neighbor,gdp_per_capita_neighbor,eco_activity_rate_neighbor,unemploy_rate_neighbor
postcode,,,,,,,,,,,,,,,,,,,,,
1000,95.0,1674.0,108.0,29.157216,275.305556,29733.0,1347.0,"['1010', '1055', '1073', '1066', '1052', '1053...",Southwest,3.704057,...,3643.666667,5055.500000,2807.166667,4518.000000,1085.166667,670.666667,295.333333,78021.0,62.661725,7.273163
1003,710.0,4034.0,38.0,1.705730,1110.078947,42183.0,8132.0,"['1007', '1005', '1006', '1004']",Southwest,22.277846,...,9966.000000,11866.500000,5846.250000,13972.250000,1116.000000,136.000000,608.250000,78021.0,62.661725,7.273163
1004,1642.0,12892.0,56.0,4.779186,688.732143,38569.0,3344.0,"['1018', '1008', '1003', '1007', '1005']",Southwest,11.717476,...,7631.600000,8804.600000,4843.000000,10358.600000,979.400000,223.600000,455.200000,78021.0,62.661725,7.273163
1005,804.0,6368.0,36.0,3.340990,826.277778,29746.0,7860.0,"['1010', '1009', '1018', '1012', '1003', '1011...",Southwest,10.775250,...,8478.428571,10418.571429,5515.000000,11381.714286,1127.285714,259.571429,557.285714,78021.0,62.661725,7.273163
1006,1304.0,8117.0,43.0,14.187561,542.348837,23321.0,3249.0,"['1007', '1003', '1005', '1009']",Southwest,3.030824,...,7395.250000,9079.000000,4735.500000,10009.750000,1141.000000,240.250000,561.750000,78021.0,62.661725,7.273163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9107,93.0,873.0,37.0,134.703815,21.837838,808.0,84.0,"['9658', '9651', '9104', '9105', '9633', '9064...",East,0.274677,...,775.666667,1376.333333,660.000000,849.333333,709.666667,289.666667,75.333333,67341.0,63.638848,2.737149
9410,159.0,1623.0,25.0,17.155476,66.360000,1659.0,356.0,"['9427', '9044', '9413', '9411', '9038', '9425...",East,1.457261,...,633.800000,1049.200000,546.000000,722.400000,1162.000000,322.000000,64.200000,67341.0,63.638848,2.737149
9411,21.0,209.0,32.0,10.993068,18.437500,590.0,78.0,"['9413', '9450', '9437', '9410', '9442', '9445']",East,2.910925,...,2156.000000,3369.000000,1907.000000,2428.000000,317.000000,726.000000,278.000000,67341.0,63.638848,2.737149


In [13]:
X_cols_n = [
    'num_stops', 'stop_density', 'avg_daily_frequency', 'max_stop_frequency',
    'population', 'male_population', 'swiss_citizen', 'married', 'age_20_64',
    'gdp_per_capita', 'eco_activity_rate', 'unemploy_rate',
    'total_residential_housing', 'pure_residential_single', 'pure_residential_multiple',
    'language',
    'num_stops_neighbor', 'stop_density_neighbor', 'avg_daily_frequency_neighbor', 'max_stop_frequency_neighbor',
    'population_neighbor', 'male_population_neighbor', 'swiss_citizen_neighbor', 'married_neighbor', 'age_20_64_neighbor',
    'gdp_per_capita_neighbor', 'eco_activity_rate_neighbor', 'unemploy_rate_neighbor',
    'total_residential_housing_neighbor', 'pure_residential_single_neighbor','pure_residential_multiple_neighbor',
]

X_train, y_train = train_df[X_cols_n], train_df[['GA_pct', 'HTA_pct']]
X_test, y_test = test_df[X_cols_n], test_df[['GA_pct', 'HTA_pct']]

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = y_train
y_test_scaled = y_test

In [15]:
from sklearn.impute import SimpleImputer

# impute missing values with mean
imputer = SimpleImputer(strategy='mean')
X_train_scaled_imputed = imputer.fit_transform(X_train_scaled)
X_test_scaled_imputed = imputer.transform(X_test_scaled)

lrg = LinearRegression()
lrg.fit(X_train_scaled_imputed, y_train_scaled)
evaluate(lrg, X_test_scaled_imputed, y_test_scaled, "LinearRegressor")

LinearRegressor R²: 0.0768, MAE: 4.06, RMSE: 6.10


In [16]:
cat_model = MultiOutputRegressor(CatBoostRegressor(verbose=0, random_state=88))
cat_model.fit(X_train_scaled_imputed, y_train_scaled)
evaluate(cat_model, X_test_scaled_imputed, y_test_scaled, "CatBoost")

CatBoost R²: 0.2205, MAE: 4.09, RMSE: 6.05


In [17]:
X_cols_no = [
    'num_stops_neighbor', 'stop_density_neighbor', 'avg_daily_frequency_neighbor', 'max_stop_frequency_neighbor',
    'population_neighbor', 'male_population_neighbor', 'swiss_citizen_neighbor', 'married_neighbor', 'age_20_64_neighbor',
    'gdp_per_capita_neighbor', 'eco_activity_rate_neighbor', 'unemploy_rate_neighbor',
    'total_residential_housing_neighbor', 'pure_residential_single_neighbor','pure_residential_multiple_neighbor',
]


X_train, y_train = train_df[X_cols_no], train_df[['GA_pct', 'HTA_pct']]
X_test, y_test = test_df[X_cols_no], test_df[['GA_pct', 'HTA_pct']]

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = y_train
y_test_scaled = y_test

In [18]:
imputer = SimpleImputer(strategy='mean')
X_train_scaled_imputed = imputer.fit_transform(X_train_scaled)
X_test_scaled_imputed = imputer.transform(X_test_scaled)

lrg = LinearRegression()
lrg.fit(X_train_scaled_imputed, y_train_scaled)
evaluate(lrg, X_test_scaled_imputed, y_test_scaled, "LinearRegressor")

cat_model = MultiOutputRegressor(CatBoostRegressor(verbose=0, random_state=88))
cat_model.fit(X_train_scaled_imputed, y_train_scaled)
evaluate(cat_model, X_test_scaled_imputed, y_test_scaled, "CatBoost")

LinearRegressor R²: 0.0739, MAE: 4.27, RMSE: 6.39
CatBoost R²: -0.0336, MAE: 4.82, RMSE: 7.10
